In [1]:
'''从零开始实现线性回归'''
import random
import torch
#构造数据集
def synthetic_data(w,b,num):
    #生成均值为0，标准差为1的随机数，大小为num，列数为len(w)（即得到num组数据）
    x = torch.normal(0,1,(num,len(w)))
    #令y = wx+b,matmul()为矩阵乘法运算
    y = torch.matmul(x,w) + b
    #加入噪音项（和y同形状的均值为0，标准差为0.01的张量）
    y += torch.normal(0,0.01,y.shape)
    #返回x以及y被重塑成列向量后的新张量
    return x,y.reshape((-1,1))

#定义给定的w和b
true_w = torch.tensor([2,-3.4])
true_b = 4.2
#通过上述函数运算得到特征与标签，并打印第1行
features,labels = synthetic_data(true_w,true_b,1000)
print('features',features[0],'labels',labels[0])

features tensor([-1.2227,  0.4074]) labels tensor([0.3805])


In [2]:
#接收小批量
def data_iter(batch_size,features,labels):
    #得到features的个数
    num = len(features)
    #生成index
    indices = list(range(num))
    #随机打乱样本
    random.shuffle(indices)
    #取出从i到i+batch_size的数据
    for i in range(0,num,batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size,num)])
        #返回当前小批量的特征和标签
        yield features[batch_indices],labels[batch_indices]

#定义批量大小
batch_size = 10
for x,y in data_iter(batch_size,features,labels):
    print(x)
    print(y)
    break

tensor([[-0.0463,  0.9305],
        [ 0.9477, -1.7878],
        [ 0.2810,  1.7328],
        [ 0.8919, -0.7689],
        [-1.4746,  0.1416],
        [ 1.4521,  0.6135],
        [-0.6669,  0.9688],
        [ 0.4359,  1.6191],
        [-0.1747, -0.9128],
        [ 0.6023, -1.2138]])
tensor([[ 0.9377],
        [12.1650],
        [-1.1264],
        [ 8.6071],
        [ 0.7795],
        [ 5.0242],
        [-0.4269],
        [-0.4411],
        [ 6.9444],
        [ 9.5115]])


In [3]:
w = torch.normal(0,0.01,size=(2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

def linreg(x,w,b):
    '''线性回归模型'''
    return torch.matmul(x,w) + b

print(w)
print(b)

tensor([[-0.0126],
        [ 0.0018]], requires_grad=True)
tensor([0.], requires_grad=True)


In [4]:
def squared_loss(y_hat,y):
    '''均方损失'''
    return (y_hat - y.reshape(y_hat.shape))**2/2

In [7]:
def sgd(params,lr,batch_size):
    '''小批量随机梯度下降'''
    #此处禁用梯度计算，节省内存。所需的梯度会在调用前由其他代码计算完成
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad/batch_size
            param.grad.zero_()

In [6]:
#定义学习率，训练次数，模型类型，损失函数类型
lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

#for循环训练
for epoch in range(num_epochs):
    #获取批量的x（特征）和y（标签）
    for x,y in data_iter(batch_size,features,labels):
        #计算损失l
        l = loss(net(x,w,b),y)
        #化l为标量后计算梯度
        l.sum().backward()
        #调用sgd（随机梯度下降）更新参数
        sgd([w,b],lr,batch_size)
    #输出每轮训练得到的损失值(with no_grad()防止重复计算占用内存)
    with torch.no_grad():
        train_l = loss(net(features,w,b),labels)
        print(f'epoch {epoch+1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.038722
epoch 2, loss 0.000140
epoch 3, loss 0.000049
